In [1]:

import torch
from trainer import Trainer
from graphBuilder import BipartiteGraph
from gnn4cd_model import GNN4CD_Model
import json 

device= torch.device("cuda" if torch.cuda.is_available() else "mps")
print("Device: ", device)

/opt/anaconda3/envs/lab4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device:  mps


In [2]:

with open("/Users/emely/Uni/Lab4/Lab4/data_corrdiff_mini/stats.json", "r") as f:
    stats = json.load(f)


In [3]:
dataset = BipartiteGraph(
        netcdf_path="/Users/emely/Uni/Lab4/Lab4/data_corrdiff_mini/hrrr_mini_train.nc", 
        stats=stats, 
        seq_l = 10,
        coarse_shape=(8,8),
        fine_shape=(64,64),
        neighbors=4
)

data = dataset[1]
data=data.to(device)
print(data)
print("Low nodes:", data['low'].x.shape)
print("High nodes:", data['high'].x.shape)
print("Low->High edges:", data[('low','to','high')].edge_index.shape)
print("High->High edges:", data[('high','within','high')].edge_index.shape)


/Users/emely/Uni/Lab4/Lab4/graphBuilder.py:54: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  self.n_samples = self.ds_input.dims['sample']


HeteroData(
  num_nodes_dict={
    low=64,
    high=4096,
  },
  idx=1,
  t=[1],
  low={
    x=[64, 10, 26],
    pos=[64, 2],
  },
  high={
    x=[4096, 4],
    y=[4096, 4],
    pos=[4096, 2],
    train_mask=[4096],
  },
  (low, to, high)={ edge_index=[2, 16384] },
  (high, within, high)={ edge_index=[2, 36100] }
)
Low nodes: torch.Size([64, 10, 26])
High nodes: torch.Size([4096, 4])
Low->High edges: torch.Size([2, 16384])
High->High edges: torch.Size([2, 36100])


In [ ]:

model = GNN4CD_Model(
    encoding_dim = 32,
    n_input_features = 26,  
    h_hid = 32,  
    n_layers = 2,
    high_in = 4,  
    low2high_out = 32,
    high_out = 32,
    n_output_vars = 4
).to(device)

In [16]:
print(len(dataset))

100000


In [5]:
subset_size = 5000  
subset_dataset = torch.utils.data.Subset(dataset, list(range(len(dataset)-subset_size, len(dataset))))

trainer = Trainer(model, subset_dataset, batch_size=10, lr=0.001, device=device)
trainer.train(epochs=10)




Epoch 1/10


Validation: 100%|██████████| 50/50 [00:21<00:00,  2.33it/s]


Train Loss: 0.590245, Val Loss: 0.509316

Epoch 2/10


Validation: 100%|██████████| 50/50 [00:21<00:00,  2.36it/s]


Train Loss: 0.388369, Val Loss: 0.307732

Epoch 3/10


...

Train Loss: 0.283326, Val Loss: 0.273319

Epoch 9/10


Validation: 100%|██████████| 50/50 [00:21<00:00,  2.38it/s]


Train Loss: 0.276775, Val Loss: 0.251349

Epoch 10/10


Validation: 100%|██████████| 50/50 [00:21<00:00,  2.38it/s]

Train Loss: 0.274680, Val Loss: 0.249572


In [5]:
from train_test import Trainer, Tester
from torch_geometric.loader import DataLoader
import torch.nn as nn
from accelerate import Accelerator
from lossFunctions import quantized_loss

train_size = int(0.9 * len(dataset))
val_size = int(0.05 * len(dataset))
test_size = len(dataset) - train_size - val_size 

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

trainer = Trainer()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

accelerator = Accelerator(mixed_precision="fp16")
model, optimizer, train_loader, val_loader = accelerator.prepare(
    model, optimizer, train_loader, val_loader
)

args = type('', (), {})() 
args.epochs = 1
args.alpha = 0.75
args.output_path = "./output/"
args.loss_fn = "MSE"

trainer.train_R_Rall(
    model = model,
    dataloader_train=train_loader,
    dataloader_val=val_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    lr_scheduler=None,
    accelerator=accelerator,
    args=args
)



Start training the regressor.

===== Epoch 1 =====
Length of DataLoader:  90000
[Epoch 0 Batch 0] Loss: 0.8702
[Epoch 0 Batch 500] Loss: 0.1052
[Epoch 0 Batch 1000] Loss: 0.4891


KeyboardInterrupt: 

TypeError: unsupported operand type(s) for -: 'slice' and 'int'